In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("/Users/baobach/ML-Projects/ml-nft-price-predictor/data/NFT_traits.csv").set_index("token_id")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 1 to 9999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   TRAITS_COUNT  9999 non-null   int64 
 1   EARRING       2976 non-null   object
 2   BACKGROUND    9999 non-null   object
 3   FUR           9999 non-null   object
 4   EYES          9999 non-null   object
 5   MOUTH         9999 non-null   object
 6   HAT           7744 non-null   object
 7   CLOTHES       8113 non-null   object
dtypes: int64(1), object(7)
memory usage: 703.1+ KB


In [9]:
df.nunique()

TRAITS_COUNT     4
EARRING          6
BACKGROUND       8
FUR             19
EYES            23
MOUTH           33
HAT             36
CLOTHES         43
dtype: int64